In [44]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from sklearn.metrics import classification_report
import tensorflow as tf
from tensorflow.keras.utils import load_img
from tqdm.notebook import tqdm
import glob

import warnings
warnings.filterwarnings('ignore')

import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'

In [11]:
train = pd.read_csv('/kaggle/input/plant-pathology-2020-fgvc7/train.csv')
train.head()

,image_id,healthy,multiple_diseases,rust,scab
0,Train_0,0,0,0,1
1,Train_1,0,1,0,0
2,Train_2,1,0,0,0
3,Train_3,0,0,1,0
4,Train_4,1,0,0,0


In [18]:
X = []
y = []
for image in tqdm(glob.glob('/kaggle/input/plant-pathology-2020-fgvc7/images/Train_*')):
    img = np.array(load_img(image,target_size = (224,224)))
    X.append(img)
    id = image.split('/')[-1].split(".")[0]
    y.append(train.loc[train['image_id'] == id,['healthy',"multiple_diseases",'rust','scab']])

  0%|          | 0/1821 [00:00<?, ?it/s]

In [19]:
X = np.array(X)
y = np.array(y).reshape(1821,4)

In [20]:
classes = list(train.columns[1:])
classes

['healthy', 'multiple_diseases', 'rust', 'scab']

In [21]:
cut = round(X.shape[0]*0.8)
cut

1457

In [22]:
X_train = X[:cut]
X_test = X[cut:]
y_train = y[:cut]
y_test = y[cut:]

In [25]:
model = tf.keras.models.load_model('/kaggle/input/model-plant/my_best_model.epoch98-loss0.32.hdf5')

In [33]:
preds = model.predict(X_test)

In [34]:
preds = preds.argmax(axis = 1)

In [37]:
y_test.shape , preds.shape

((364, 4), (364,))

In [40]:
y_test = y_test.argmax(axis = 1)

In [41]:
y_test[0]

0

In [39]:
preds[0]

0

In [45]:
print(classification_report(y_test,preds))

              precision    recall  f1-score   support

           0       0.90      0.93      0.91        97
           1       0.00      0.00      0.00        17
           2       0.93      0.98      0.95       127
           3       0.90      0.94      0.92       123

    accuracy                           0.91       364
   macro avg       0.68      0.71      0.70       364
weighted avg       0.87      0.91      0.89       364

